In [4]:
from pysndfx import AudioEffectsChain
import wubwub as wb
import wubwub.sounds as snd

SYNTH = snd.load('synth.elka')
BASS = snd.load('bass.synth')
DRUMS = snd.load('drums.house')

seq = wb.Sequencer(bpm=110, beats=32)

arp = seq.add_arpeggiator(sample=SYNTH['C3'], name='arp', basepitch='C3', freq=1/6, method='updown')
C = wb.chord_from_name(root='C2', lengths=8) + wb.chord_from_name(root='C3', add=12)
E = wb.chord_from_name(root='E2', lengths=8) + wb.chord_from_name(root='E3', add=12)
F = wb.chord_from_name(root='F2', lengths=8) + wb.chord_from_name(root='F3', add=12)
Fm = wb.chord_from_name(root='F2', kind='m', lengths=8) + wb.chord_from_name(root='F3', kind='m', add=12)
arp[1] = C
arp[9] = E
arp[17] = F
arp[25] = Fm
arp.effects = AudioEffectsChain().reverb(reverberance=10, wet_gain=1).lowpass(5000)

lead1 = seq.add_sampler(sample=SYNTH['C3'], name='lead1', basepitch='C3')
melodypat = wb.Pattern([1,3,4,5,7,9], length=16)
lead1.make_notes(beats=melodypat, pitches=[4, 2, 0, 7, 4, 8], lengths=[2,2,2,2,2,8])
lead1.make_notes(beats=melodypat.onmeasure(2), pitches=[9, 7, 5, 4, 0, 2], lengths=[2,2,2,2,2,8])
lead1.effects = (AudioEffectsChain()
                 .reverb(room_scale=100, wet_gain=4, reverberance=40)
                 .delay(delays=[500,1000,1500,2000], decays=[.7])
                 .lowpass(6000))
lead1.volume += 5

lead2 = seq.duplicate_track('lead1', newname='lead2')
lead2notes = {b:n.alter(pitch=n.pitch-.4) for b, n in lead1.notedict.items()}
lead2.add_fromdict(lead2notes)

bass = seq.add_sampler(sample=BASS['fat8tone'], name='bass', basepitch='C3')
bass[[1,5]] = wb.Note(pitch='C3', length=4)
bass[[9,13]] = wb.Note(pitch='E3', length=4)
bass[[17, 21, 25, 29]] = wb.Note(pitch='F3', length=4)
bass.volume -= 12
bass.effects = AudioEffectsChain().overdrive(colour=50).reverb().lowpass(1000)

kick = seq.add_sampler(sample=DRUMS['kick7'], name='kick')
kick.make_notes_every(4)
kick.make_notes_every(4, offset=-.5)
kick.clean()
kick.effects = AudioEffectsChain().reverb().lowpass(600)
kick.volume -= 4

seq.build(overhang=4)